# Kubeflow Fairing Introduction

Kubeflow Fairing is a Python package that streamlines the process of `building`, `training`, and `deploying` machine learning (ML) models in a hybrid cloud environment. By using Kubeflow Fairing and adding a few lines of code, you can run your ML training job locally or in the cloud, directly from Python code or a Jupyter notebook. After your training job is complete, you can use Kubeflow Fairing to deploy your trained model as a prediction endpoint.


# How does Kubeflow Fairing work

Kubeflow Fairing 
1. Packages your Jupyter notebook, Python function, or Python file as a Docker image
2. Deploys and runs the training job on Kubeflow or AI Platform. 
3. Deploy your trained model as a prediction endpoint on Kubeflow after your training job is complete.


# Goals of Kubeflow Fairing project

- Easily package ML training jobs: Enable ML practitioners to easily package their ML model training code, and their code’s dependencies, as a Docker image.
- Easily train ML models in a hybrid cloud environment: Provide a high-level API for training ML models to make it easy to run training jobs in the cloud, without needing to understand the underlying infrastructure.
- Streamline the process of deploying a trained model: Make it easy for ML practitioners to deploy trained ML models to a hybrid cloud environment.

## Train and deploy model on Kubeflow in Notebooks

This examples comes from a upstream fairing [example](https://github.com/kubeflow/fairing/tree/master/examples/prediction).


Please check Kaggle competiton [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)
for details about the ML problem we want to resolve.

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Amazon EKS. This notebook demonstrate how to:

* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow,
* Call the deployed endpoint for predictions.


### Install python dependencies

In [1]:
# Install latest Fairing from github repository
!pip install kubeflow-fairing==0.7.1

You are using pip version 19.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%%writefile requirements.txt
pandas
joblib
numpy
xgboost
scikit-learn>=0.21.0
seldon-core
tornado>=6.0.3

Overwriting requirements.txt


In [3]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 102kB 7.3MB/s a 0:00:011
    100% |████████████████████████████████| 92kB 45.8MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 41.4MB/s ta 0:00:011
    100% |████████████████████████████████| 81kB 48.5MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 40.6MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 42.8MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 40.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 37.5MB/s ta 0:00:01


  Stored in directory: /root/.cache/pip/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Stored in directory: /root/.cache/pip/wheels/f2/84/7f/e89da3ee8ce35598d6382b6389fa2ada5d66acca2422537994
  Stored in directory: /root/.cache/pip/wheels/93/e9/b5/1cdc3544f99a54caca13832b5afa26fd98701fe709dc049576
  Stored in directory: /root/.cache/pip/wheels/d7/7a/30/d212623a4cd34f6cce400f8122b1b7af740d3440c68023d51f
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built Flask-OpenTracing jaeger-client opentracing threadloop thrift
kfp 0.1 has requirement google-auth==1.6.1, but you'll have google-auth 1.11.0 which is incompatible.
kfp 0.1 has requirement google-cloud-storage==1.13.0, but you'll have google-cloud-storage 1.25.0 which is incompatible.
kfp 0.1 has requirement kubernetes==8.0.0, but you'll have kubernetes 10.0.1 which is incompatible.
google-cloud-bigquery 1.17.1 has requirement google-resumable

In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Train on Kubernetes

We will show you how to run a training job in kubernetes cluster. You can use `ECR` as your container image registry.

In [1]:
# Authenticate ECR
# This command retrieves a token that is valid for a specified registry for 12 hours, 
# and then it prints a docker login command with that authorization token. 
# Then we executate this command to login ECR

REGION='us-west-2'
!eval $(aws ecr get-login --no-include-email --region=$REGION)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [2]:
# Create an ECR repository in the same region
!aws ecr describe-repositories --repository-names fairing-job --region=$REGION || aws ecr create-repository --repository-name fairing-job --region=$REGION

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:903253828154:repository/fairing-job",
            "registryId": "903253828154",
            "repositoryName": "fairing-job",
            "repositoryUri": "903253828154.dkr.ecr.us-west-2.amazonaws.com/fairing-job",
            "createdAt": 1587849943.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


### Develop your model

In [3]:
import argparse
import logging
import joblib
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

In [4]:
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds" %
               (model.best_score,
                model.best_iteration+1))
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    logging.info("mean_absolute_error=%.2f", mean_absolute_error(predictions, test_y))

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)
    
    
class HousingServe(object):
    
    def __init__(self):
        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names=None):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction

### Train an XGBoost model in a notebook

In [5]:
model = HousingServe()
model.train()

[21:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:177514
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161858
[2]	validation_0-rmse:147237
[3]	validation_0-rmse:134132
[4]	validation_0-rmse:122224
[5]	validation_0-rmse:111538
[6]	validation_0-rmse:102142
[7]	validation_0-rmse:93392.3
[8]	validation_0-rmse:85824.6
[9]	validation_0-rmse:79667.6
[10]	validation_0-rmse:73463.4
[11]	validation_0-rmse:68059.4
[12]	validation_0-rmse:63350.5
[13]	validation_0-rmse:59732.1
[14]	validation_0-rmse:56260.7
[15]	validation_0-rmse:53392.6
[16]	validation_0-rmse:50770.8
[17]	validation_0-rmse:48107.8
[18]	validation_0-rmse:45923.9
[19]	validation_0-rmse:44154.2
[20]	validation_0-rmse:42488.1
[21]	validation_0-rmse:41263.3
[22]	validation_0-rmse:40212.8
[23]	validation_0-rmse:39089.1
[24]	validation_0-rmse:37691.1
[25]	validation_0-rmse:36875.2
[26]	validation_0-r

mean_absolute_error=18173.15
Model export success: trained_ames_model.dat


Best RMSE on eval: 28787.72 with 50 rounds


### Set up Kubeflow Fairing for training and predictions



In [6]:
from kubeflow import fairing
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import KubeflowAWSBackend


from kubeflow import fairing

FAIRING_BACKEND = 'KubeflowAWSBackend'

AWS_ACCOUNT_ID = fairing.cloud.aws.guess_account_id()
AWS_REGION = 'us-west-2'
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)

S3_BUCKET = 'sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)


In [7]:
import importlib

if FAIRING_BACKEND == 'KubeflowAWSBackend':
    from kubeflow.fairing.builders.cluster.s3_context import S3ContextSource
    BuildContext = S3ContextSource(
        aws_account=AWS_ACCOUNT_ID, region=AWS_REGION,
        bucket_name=S3_BUCKET
    )

BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), FAIRING_BACKEND)

### Train an XGBoost model on Kubeflow
Import the `TrainJob` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on Kubeflow.


In [8]:
from kubeflow.fairing import TrainJob
train_job = TrainJob(HousingServe, input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=BackendClass(build_context_source=BuildContext))
train_job.submit()

Using default base docker image: registry.hub.docker.com/library/python:3.6.8
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_by7evs2g
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-5mz5n-k9hfg to start...
Waiting for fairing-builder-5mz5n-k9hfg to start...
Waiting for fairing-builder-5mz5n-k9hfg to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0003] Unpacking rootfs as cm

  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/1c/54/34/fd47cd9b308826cc4292b54449c1899a30251ef3b506bc91ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/93/e9/b5/1cdc3544f99a54caca13832b5afa26fd98701fe709dc049576
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/f2/84/7f/e89da3ee8ce35598d6382b6389fa2ada5d66acca2422537994
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/d7/7a/30/d212623a4cd34f6cce400f8122b1b7af740d3440c68023d51f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qapdv_x/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff3

You should consider upgrading via the 'pip install --upgrade pip' command.
INFO[0053] Taking snapshot of full filesystem...
INFO[0067] Using files from context: [/kaniko/buildcontext/app]
INFO[0067] COPY /app/ /app/
INFO[0067] Taking snapshot of files...


The job fairing-job-c2n5j launched.
Waiting for fairing-job-c2n5j-8ql9c to start...
Waiting for fairing-job-c2n5j-8ql9c to start...
Waiting for fairing-job-c2n5j-8ql9c to start...
Pod started running True


[0]	validation_0-rmse:177565.32812
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161967.20312
[2]	validation_0-rmse:148001.90625
[3]	validation_0-rmse:135010.15625
[4]	validation_0-rmse:123514.68750
[5]	validation_0-rmse:113210.39062
[6]	validation_0-rmse:103914.61719
[7]	validation_0-rmse:95352.98438
[8]	validation_0-rmse:87878.78125
[9]	validation_0-rmse:81683.14062
[10]	validation_0-rmse:75828.78906
[11]	validation_0-rmse:70085.50781
[12]	validation_0-rmse:65076.06641
[13]	validation_0-rmse:60899.81641
[14]	validation_0-rmse:57354.23047
[15]	validation_0-rmse:54106.52734
[16]	validation_0-rmse:51402.43359
[17]	validation_0-rmse:48774.05078
[18]	validation_0-rmse:46360.19141
[19]	validation_0-rmse:44304.82422
[20]	validation_0-rmse:42618.65234
[21]	validation_0-rmse:41219.89062
[22]	validation_0-rmse:39885.14453
[23]	validation_0-rmse:38977.96484
[24]	validation_0-rmse:37856.48047
[25]	validation_0-rmse:36739.78125
[26]	validation_0-rmse:35847

Cleaning up job fairing-job-c2n5j...


'fairing-job-c2n5j'